In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import emoji
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-data/neg_tweets.csv
/kaggle/input/twitter-data/pos_tweets.csv


In [15]:
#Import dataset and view sample
pos=pd.read_csv('/kaggle/input/twitter-data/pos_tweets.csv')
neg=pd.read_csv('/kaggle/input/twitter-data/neg_tweets.csv')
print(neg.head(),'\n\n# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #\n\n',pos.head())

                                    user_description
0  Food addict, sweet tooth, house music and tequ...
1  Bollywood and Celebrities Brands Breaking News...
2          ‏‏‏حساب شخصي. مدمن كرة قدم. CFC #Ittihad#
3                                   IG: _RichyRozay_
4                                    fuck em we ball 

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

                                     user_description
0                                            alright
1  ADHD & Diverse-Ability Transition Coach/Traine...
2  queer trans bot | she/her | property of @Corde...
3                 People tend to say I'm stone cold.
4  Hi, I'm Kris. I'm an okay cross/cosplayer. he/...


In [16]:
#Define helper functions and preprocessing function
def load_dict_smileys():
    
    return {
        ":‑)":"smiley",
        ":-]":"smiley",
        ":-3":"smiley",
        ":->":"smiley",
        "8-)":"smiley",
        ":-}":"smiley",
        ":)":"smiley",
        ":]":"smiley",
        ":3":"smiley",
        ":>":"smiley",
        "8)":"smiley",
        ":}":"smiley",
        ":o)":"smiley",
        ":c)":"smiley",
        ":^)":"smiley",
        "=]":"smiley",
        "=)":"smiley",
        ":-))":"smiley",
        ":‑D":"smiley",
        "8‑D":"smiley",
        "x‑D":"smiley",
        "X‑D":"smiley",
        ":D":"smiley",
        "8D":"smiley",
        "xD":"smiley",
        "XD":"smiley",
        ":‑(":"sad",
        ":‑c":"sad",
        ":‑<":"sad",
        ":‑[":"sad",
        ":(":"sad",
        ":c":"sad",
        ":<":"sad",
        ":[":"sad",
        ":-||":"sad",
        ">:[":"sad",
        ":{":"sad",
        ":@":"sad",
        ">:(":"sad",
        ":'‑(":"sad",
        ":'(":"sad",
        ":‑P":"playful",
        "X‑P":"playful",
        "x‑p":"playful",
        ":‑p":"playful",
        ":‑Þ":"playful",
        ":‑þ":"playful",
        ":‑b":"playful",
        ":P":"playful",
        "XP":"playful",
        "xp":"playful",
        ":p":"playful",
        ":Þ":"playful",
        ":þ":"playful",
        ":b":"playful",
        "<3":"love"
        }

# self defined contractions
def load_dict_contractions():
    
    return {
        "ain't":"is not",
        "amn't":"am not",
        "aren't":"are not",
        "can't":"cannot",
        "'cause":"because",
        "couldn't":"could not",
        "couldn't've":"could not have",
        "could've":"could have",
        "daren't":"dare not",
        "daresn't":"dare not",
        "dasn't":"dare not",
        "didn't":"did not",
        "doesn't":"does not",
        "don't":"do not",
        "e'er":"ever",
        "em":"them",
        "everyone's":"everyone is",
        "finna":"fixing to",
        "gimme":"give me",
        "gonna":"going to",
        "gon't":"go not",
        "gotta":"got to",
        "hadn't":"had not",
        "hasn't":"has not",
        "haven't":"have not",
        "he'd":"he would",
        "he'll":"he will",
        "he's":"he is",
        "he've":"he have",
        "how'd":"how would",
        "how'll":"how will",
        "how're":"how are",
        "how's":"how is",
        "i'd":"i would",
        "i'll":"i will",
        "i'm":"i am",
        "i'm'a":"i am about to",
        "i'm'o":"i am going to",
        "isn't":"is not",
        "it'd":"it would",
        "it'll":"it will",
        "it's":"it is",
        "i've":"i have",
        "kinda":"kind of",
        "let's":"let us",
        "mayn't":"may not",
        "may've":"may have",
        "mightn't":"might not",
        "might've":"might have",
        "mustn't":"must not",
        "mustn't've":"must not have",
        "must've":"must have",
        "needn't":"need not",
        "ne'er":"never",
        "o'":"of",
        "o'er":"over",
        "ol'":"old",
        "oughtn't":"ought not",
        "shalln't":"shall not",
        "shan't":"shall not",
        "she'd":"she would",
        "she'll":"she will",
        "she's":"she is",
        "shouldn't":"should not",
        "shouldn't've":"should not have",
        "should've":"should have",
        "somebody's":"somebody is",
        "someone's":"someone is",
        "something's":"something is",
        "that'd":"that would",
        "that'll":"that will",
        "that're":"that are",
        "that's":"that is",
        "there'd":"there would",
        "there'll":"there will",
        "there're":"there are",
        "there's":"there is",
        "these're":"these are",
        "they'd":"they would",
        "they'll":"they will",
        "they're":"they are",
        "they've":"they have",
        "this's":"this is",
        "those're":"those are",
        "'tis":"it is",
        "'twas":"it was",
        "wanna":"want to",
        "wasn't":"was not",
        "we'd":"we would",
        "we'd've":"we would have",
        "we'll":"we will",
        "we're":"we are",
        "weren't":"were not",
        "we've":"we have",
        "what'd":"what did",
        "what'll":"what will",
        "what're":"what are",
        "what's":"what is",
        "what've":"what have",
        "when's":"when is",
        "where'd":"where did",
        "where're":"where are",
        "where's":"where is",
        "where've":"where have",
        "which's":"which is",
        "who'd":"who would",
        "who'd've":"who would have",
        "who'll":"who will",
        "who're":"who are",
        "who's":"who is",
        "who've":"who have",
        "why'd":"why did",
        "why're":"why are",
        "why's":"why is",
        "won't":"will not",
        "wouldn't":"would not",
        "would've":"would have",
        "y'all":"you all",
        "you'd":"you would",
        "you'll":"you will",
        "you're":"you are",
        "you've":"you have",
        "whatcha":"what are you",
        "luv":"love",
        "sux":"sucks"
        }

def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

def clean_str(string):

    string = re.sub(r'http\S+', '', string, flags=re.MULTILINE)
    string = re.sub(r'www.\S+', '', string, flags=re.MULTILINE)
#    string = re.sub(r'@\S+', '', string, flags=re.MULTILINE)
    string = re.sub(r"http\S+", "", string, flags=re.MULTILINE)
#    print(string)
    string = string.replace('\n', ' ').replace('\t', ' ')
    string = string.lower()
    string = reduce_lengthening(string)
    
    #CONTRACTIONS source: https://en.wikipedia.org/wiki/Contraction_%28grammar%29
    CONTRACTIONS = load_dict_contractions()
    string = string.replace("’","'").replace("“","'").replace("”","'").replace("’","'").replace("‘","'")
    words = string.split()
    reformed = [CONTRACTIONS[word] if word in CONTRACTIONS else word for word in words]
    string = " ".join(reformed)

    #Deal with emoticons source: https://en.wikipedia.org/wiki/List_of_emoticons
    SMILEY = load_dict_smileys()  
    words = string.split()
    reformed = [SMILEY[word] if word in SMILEY else word for word in words]
    string = " ".join(reformed)
    
    #Deal with emojis
    string = emoji.demojize(string)

    string = ' '.join(string.split())

    string = re.sub('[^A-Za-z0-9.?;!]+', ' ', string).lstrip().lower()
    string = string.replace(";"," ; ").replace("."," . ").replace("?"," ? ").replace("!"," ! ")

    string = ' '.join(string.split())    

#    load()
#    segs = segment(string)
#    print(segs)
#    string = ' '.join(segs)
#    print(string)

    return string.strip().lower()

In [17]:
#Apply text cleaning to both datasets
neg['user_description']=neg['user_description'].apply(lambda x: clean_str(x))
pos['user_description']=pos['user_description'].apply(lambda x: clean_str(x))

#Create label column (0 for neg and 1 for pos)
neg['label']=0
pos['label']=1

#Combine the 2 datasets
df=[neg,pos]
df=pd.concat(df)
df

#Split it into train and test with shuffling
test,train=train_test_split(df,test_size=0.25, random_state=1234, shuffle=True)

#View the results
print(train,test,sep='\n')

                                       user_description  label
3317            want to try something new in your bed ?      0
402   books wrestling music ganja and animals . get ...      1
3274  jon snow . lord of the north . swordsman leade...      0
4401  born feb2 1992 seahawks fan run what you broun...      1
545                                               ! ! !      0
...                                                 ...    ...
3318  writer journalist mental health campaigner . c...      1
3185  nintendo switch going to be lit spare tony shh...      0
2761  in the quiet words of the virgin mary come aga...      1
217   follow my site csa survivor supports end to dv...      1
4048                rainbow flag love wins rainbow flag      0

[2650 rows x 2 columns]
                                       user_description  label
5279  queer trans bot she her property of cordelia e...      1
2255  23 programmer internetislife i am just trying ...      0
5038  we are all going to be o

In [20]:
# Load pre-trained model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
# Tokenize input text
text = "Hello, how are you doing today?"
inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")

# Run model inference
outputs = model(**inputs)

# Get predicted class
_, predicted_class = torch.max(outputs.logits, dim=1)
print(predicted_class)




#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

#model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")

OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.